<a href="https://colab.research.google.com/github/apirakqqqqq/Pan-sharpening/blob/main/Apirak__Pan_sharpening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [ ]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-apirakqqqq') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [ ]:
# กำหนดพื้นที่สนใจ
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [ ]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [ ]:
# สร้างแผนที่
Map = geemap.Map(center=[18.84423947416328, 98.95799098999555], zoom=14)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร


**ตอบข้อ 1** การเพิ่มความสามารถในการแยกแยะรายละเอียดเชิงพื้นที่ (spatial detail) ของวัตถุบนผิวโลก โดยอาศัยข้อมูลจากแบนด์แพนโครมาติก เพื่อให้ภาพหลายสเปกตรัมสามารถแสดงโครงสร้างพื้นที่ได้ชัดเจนขึ้น แต่ยังคงข้อมูลเดิลไว้ จากเดิมภาพจาก landsat9 เป็น Multispectal ความละเอียดที่ 30 m แต่การทำมาทำเป็น Pan-sharpening เปลี่ยนค่าความละเอียดเป็น 15m ทำให้เห็นโครงสร้างชัดมากขึ้น


**ตอบข้อ 2**
เทคนิคPan-sharpeningเป็นกระบวนการปรับความคมชัดของภาพถ่ายดาวเทียมโดยอาศัยการผสานข้อมูลจากภาพแพนโครมาติกซึ่งมีความละเอียดเชิงพื้นที่สูงแต่ไม่มีข้อมูลสี กับภาพหลายสเปกตรัมซึ่งมีข้อมูลสีครบถ้วนแต่มีความละเอียดเชิงพื้นที่ต่ำกว่า กระบวนการเริ่มจากการรับข้อมูลภาพทั้งสองชนิดจากดาวเทียมเดียวกัน จากนั้นต้องเตรียมข้อมูลให้พร้อมใช้งานโดยปรับแก้ตำแหน่งเชิงเรขาคณิตให้ภาพทั้งสองอยู่ในระบบพิกัดเดียวกันและมีการจัดแนวพิกเซลตรงกัน ขั้นตอนสำคัญคือกระบวนการสุ่มตัวอย่างใหม่ (resampling) ซึ่งใช้ปรับภาพหลายสเปกตรัมให้มีขนาดพิกเซลเท่ากับภาพแพนโครมาติก เพื่อให้สามารถผสานข้อมูลกันได้อย่างถูกต้อง การสุ่มตัวอย่างใหม่นี้เป็นการคำนวณค่าพิกเซลใหม่ของภาพหลายสเปกตรัมโดยอ้างอิงจากค่าพิกเซลเดิม ทำให้ภาพหลายสเปกตรัมมีความละเอียดเชิงพื้นที่สูงขึ้นในเชิงโครงสร้าง แต่ยังคงลักษณะข้อมูลสเปกตรัมไว้ หลังจากนั้นจึงนำรายละเอียดเชิงพื้นที่จากภาพแพนโครมาติกมาผสานเข้ากับข้อมูลสีของภาพหลายสเปกตรัมด้วยเทคนิค Pan-sharpening ต่าง ๆ เช่น IHS หรือ Brovey ส่งผลให้ภาพผลลัพธ์มีทั้งความคมชัดเชิงพื้นที่สูงและยังคงข้อมูลสีใกล้เคียงความเป็นจริงมากขึ้น ภาพที่ได้จึงเหมาะสำหรับการแปลภาพและการนำเสนอข้อมูลเชิงพื้นที่ เนื่องจากสามารถมองเห็นรายละเอียดของวัตถุบนพื้นผิวโลกได้ชัดเจนกว่าภาพหลายสเปกตรัมเดิม